In [1]:
from sklearn.datasets import load_iris
import pandas as pd
from pandasql import sqldf
from pandasql import load_meat, load_births
import re

births = load_births()
meat = load_meat()
iris = load_iris()
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_df['species'] = pd.Categorical.from_codes(iris.target, iris.target_names)
iris_df.columns = [re.sub("[() ]", "", col) for col in iris_df.columns]

/Users/jason.xie/anaconda/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [3]:
iris_df.head()

,sepallengthcm,sepalwidthcm,petallengthcm,petalwidthcm,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [6]:
print(sqldf("SELECT * FROM iris_df LIMIT 10;", locals()))

   sepallengthcm  sepalwidthcm  petallengthcm  petalwidthcm species
0            5.1           3.5            1.4           0.2  setosa
1            4.9           3.0            1.4           0.2  setosa
2            4.7           3.2            1.3           0.2  setosa
3            4.6           3.1            1.5           0.2  setosa
4            5.0           3.6            1.4           0.2  setosa
5            5.4           3.9            1.7           0.4  setosa
6            4.6           3.4            1.4           0.3  setosa
7            5.0           3.4            1.5           0.2  setosa
8            4.4           2.9            1.4           0.2  setosa
9            4.9           3.1            1.5           0.1  setosa


In [7]:
print(sqldf("SELECT sepalwidthcm, species FROM iris_df LIMIT 10;", locals()))

   sepalwidthcm species
0           3.5  setosa
1           3.0  setosa
2           3.2  setosa
3           3.1  setosa
4           3.6  setosa
5           3.9  setosa
6           3.4  setosa
7           3.4  setosa
8           2.9  setosa
9           3.1  setosa


In [9]:
q = """
      select
        species
        , avg(sepalwidthcm)
        , min(sepalwidthcm)
        , max(sepalwidthcm)
      from
        iris_df
      group by
        species;
        
"""
print("*" * 80) # lol this is a good way to print lines
print("aggregation")
print("-" * 80)
print(q)
print(sqldf(q, locals()))

********************************************************************************
aggregation
--------------------------------------------------------------------------------

      select
        species
        , avg(sepalwidthcm)
        , min(sepalwidthcm)
        , max(sepalwidthcm)
      from
        iris_df
      group by
        species;
        

      species  avg(sepalwidthcm)  min(sepalwidthcm)  max(sepalwidthcm)
0      setosa              3.418                2.3                4.4
1  versicolor              2.770                2.0                3.4
2   virginica              2.974                2.2                3.8


### what are locals and globals?
* [locals](https://docs.python.org/2/library/functions.html#locals)
* [globals](https://docs.python.org/2/library/functions.html#globals)

In [16]:
set(globals().keys()) - set(locals().keys())

set()

In [17]:
def pysqldf(q):
    "add this to your script if you get tired of calling locals()"
    return sqldf(q, globals())


print("*" * 80)
print("calling from a helper function")
print('''def pysqldf(q):)
    "add this to your script if you get tired of calling locals()"
        return sqldf(q, globals())''')
print("-" * 80)
print(q)
print(pysqldf(q))


********************************************************************************
calling from a helper function
def pysqldf(q):)
    "add this to your script if you get tired of calling locals()"
        return sqldf(q, globals())
--------------------------------------------------------------------------------

      select
        species
        , avg(sepalwidthcm)
        , min(sepalwidthcm)
        , max(sepalwidthcm)
      from
        iris_df
      group by
        species;
        

      species  avg(sepalwidthcm)  min(sepalwidthcm)  max(sepalwidthcm)
0      setosa              3.418                2.3                4.4
1  versicolor              2.770                2.0                3.4
2   virginica              2.974                2.2                3.8


In [18]:
q = """
    select
        a.*
    from
        iris_df a
    inner join
        iris_df b
            on a.species = b.species
    limit 10;
"""

print("*" * 80)
print("joins")
print("-" * 80)
print(q)
print(pysqldf(q))

********************************************************************************
joins
--------------------------------------------------------------------------------

    select
        a.*
    from
        iris_df a
    inner join
        iris_df b
            on a.species = b.species
    limit 10;

   sepallengthcm  sepalwidthcm  petallengthcm  petalwidthcm species
0            5.1           3.5            1.4           0.2  setosa
1            5.1           3.5            1.4           0.2  setosa
2            5.1           3.5            1.4           0.2  setosa
3            5.1           3.5            1.4           0.2  setosa
4            5.1           3.5            1.4           0.2  setosa
5            5.1           3.5            1.4           0.2  setosa
6            5.1           3.5            1.4           0.2  setosa
7            5.1           3.5            1.4           0.2  setosa
8            5.1           3.5            1.4           0.2  setosa
9            5.1

In [21]:
q = """
    select
        *
    from
        iris_df
    where
        species = 'virginica'
        and sepallengthcm > 7.2
    ORDER BY sepallengthcm DESC
    LIMIT 2;
"""
print("*" * 80)
print("where clause")
print("-" * 80)
print(q)
print(pysqldf(q))

********************************************************************************
where clause
--------------------------------------------------------------------------------

    select
        *
    from
        iris_df
    where
        species = 'virginica'
        and sepallengthcm > 7.2
    ORDER BY sepallengthcm DESC
    LIMIT 2;

   sepallengthcm  sepalwidthcm  petallengthcm  petalwidthcm    species
0            7.9           3.8            6.4           2.0  virginica
1            7.7           3.8            6.7           2.2  virginica


In [22]:
iris_df['id'] = range(len(iris_df))
q = """
    select
        *
    from
        iris_df
    where
        id in (select id from iris_df where sepalwidthcm*sepallengthcm > 25);
"""
print("*" * 80)
print("subqueries")
print("-" * 80)
print(q)
print(pysqldf(q))

********************************************************************************
subqueries
--------------------------------------------------------------------------------

    select
        *
    from
        iris_df
    where
        id in (select id from iris_df where sepalwidthcm*sepallengthcm > 25);

   sepallengthcm  sepalwidthcm  petallengthcm  petalwidthcm    species   id
0            5.7           4.4            1.5           0.4     setosa   15
1            7.2           3.6            6.1           2.5  virginica  109
2            7.7           3.8            6.7           2.2  virginica  117
3            7.9           3.8            6.4           2.0  virginica  131


In [23]:
q = """
    SELECT
        m.*
        , b.births
    FROM
        meat m
    INNER JOIN
        births b
            on m.date = b.date
    ORDER BY
        m.date;
"""

print(pysqldf(q).head())

                         date    beef  veal    pork  lamb_and_mutton  \
0  1975-01-01 00:00:00.000000  2106.0  59.0  1114.0             36.0   
1  1975-02-01 00:00:00.000000  1845.0  50.0   954.0             31.0   
2  1975-03-01 00:00:00.000000  1891.0  57.0   976.0             35.0   
3  1975-04-01 00:00:00.000000  1895.0  60.0  1100.0             34.0   
4  1975-05-01 00:00:00.000000  1849.0  59.0   934.0             31.0   

   broilers  other_chicken  turkey  births  
0     646.2            NaN    64.9  265775  
1     570.2            NaN    47.1  241045  
2     616.6            NaN    54.4  268849  
3     688.3            NaN    68.7  247455  
4     690.1            NaN    81.9  254545  
